# Imports 

In [27]:
import numpy as np
np.set_printoptions(suppress=True)
import h5py
import pandas as pd
import time
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import os
import sys
%matplotlib widget
%load_ext autoreload
%autoreload 2

from WindowedDistributionPlotter import compute_dpp_tet_tet_distribution_plot_with_fight_bouts

#sys.path.append('./lib/')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load the tracking data

Some kinematics variables can be computed from post-processed data, <br>
but for others it is best to compute from raw data, and do the post-processing afterwards. <br>
So we load both raw and smoothed data. <br>
<br>
We are also going to load arrays which give us fight regions, winner and loser, and th

In [8]:
main_load_folder = '/media/liam/hd1/fighting_data/tracking_results/'

# ----------------------#

interp_polyOrd=1  # the order of the polynomial used for interpolation
interp_limit=5    # the maximum number of frames to interpolate over
savgol_win=9      # the number of frames for the Savitzky-Golay filter
savgol_ord=2      # the polynomial order for the Savitzky-Golay filter
dt=0.01           # the frame rate of the recording

# -----------------------#

loadpaths = glob.glob(os.path.join(main_load_folder, '*results.h5'))
loadpaths.sort()

# parse the exp names
expNames = [path.split('/')[-1][:23] for path in loadpaths]

smooth_trajectories = []
for path in loadpaths:
    with h5py.File(path, 'r') as hf:
        tracks_3D_smooth = hf['tracks_3D_smooth'][:]
    smooth_trajectories.append(tracks_3D_smooth)
    
raw_trajectories = []
for path in loadpaths:
    with h5py.File(path, 'r') as hf:
        tracks_3D_raw = hf['tracks_3D_raw'][:]
    raw_trajectories.append(tracks_3D_raw)
    
    
# -----------------------------#
# load other useful information

    
other_info_loadpath = os.path.join(main_load_folder, 'winners_losers_inconclusive.h5')
with h5py.File(other_info_loadpath, 'r') as hf:
    winner_idxs = hf['winner_idxs'][:]
    loser_idxs = hf['loser_idxs'][:]
    conclusive_winner_loser = hf['conclusive_winner_loser'][:]
    already_established_dominance = hf['already_established_dominance'][:] 

    

fight_bout_load_path = os.path.join(main_load_folder, 'fightBouts.h5')
with h5py.File(fight_bout_load_path, 'r') as hf:
    refined_exps_fight_info = hf['refined_exps_fight_info'][:]           # all fight-bouts detected with refined boarders
    fight_bout_info = hf['fight_bout_info'][:]                           # the figure7 windows
    fight_bout_info_noDurThresh = hf['fight_bout_info_noDurThresh'][:]   # the figure7 windows without the size threshold (i.e. more fights

'/home/liam/temp/image_transfer/fight_detector_results/FishTank20200127_143538.png'

# Plot all detected bouts from all experiments

In [28]:
t0 = time.perf_counter()

saveFolder = '/home/liam/temp/image_transfer/refined_exps_fight_info'
if ~os.path.exists(saveFolder):
    os.makedirs(saveFolder, exist_ok=True)


for expIdx in range(len(expNames)):

    # plotting params
    expName = expNames[expIdx]                  # the name of the exp, for plot title
    figsavepath = os.path.join(saveFolder, 
                             f'{expName}.png')  # the filepath of the outputted png
    window_size=6000                            # the window width [frames]
    window_step=100                             # the step forward in time between windows [frames]
    dpp_bins = np.linspace(0, 30, 300)          # visualization bins
    tet_bins = np.linspace(-np.pi, np.pi, 300)  # visualization bins
    dpp_vmax=150                                # the max count color threshold when plotting.
    tet_vmax=150                                # the max count color threshold when plotting.
    num_xticks=10                               # the number of xaxis time ticks
    cmap='Blues'                                # colorscheme
    fps=100                                     # for frame-minutes conversions
    use_cbar=False                              # use/not-use the colorbar of the plots

    traj = smooth_trajectories[expIdx]
    expName = expNames[expIdx]
    winIdx = winner_idxs[expIdx]
    losIdx = loser_idxs[expIdx]
    has_clear_winner = conclusive_winner_loser[expIdx]
    exp_fight_bout_info = refined_exps_fight_info[refined_exps_fight_info[:,0]==expIdx][:,1:3]


    compute_dpp_tet_tet_distribution_plot_with_fight_bouts(traj, expName, exp_fight_bout_info, figsavepath, dpp_bins, tet_bins,
                                                           winnerIdx=winIdx, loserIdx=losIdx, has_clear_winner=has_clear_winner, 
                                                           window_size=window_size, window_step=window_step, dpp_vmax=dpp_vmax, 
                                                           tet_vmax=tet_vmax, num_xticks=num_xticks, cmap='Blues', fps=fps, use_cbar=use_cbar)
    print(f'{expName} finished: {time.perf_counter()-t0} s')


tE = time.perf_counter()
print()
print('finished {0}: {1} s'.format(expName, tE-t0))

FishTank20200127_143538 finished: 19.449291278840974 s
FishTank20200129_140656 finished: 35.28962769382633 s
FishTank20200130_153857 finished: 46.14810315100476 s
FishTank20200130_181614 finished: 57.41504027484916 s
FishTank20200207_161445 finished: 67.44163368991576 s
FishTank20200213_154940 finished: 82.42667682492174 s
FishTank20200214_153519 finished: 92.99783395696431 s
FishTank20200217_160052 finished: 105.69214826403186 s
FishTank20200218_153008 finished: 117.34042665781453 s
FishTank20200316_163320 finished: 137.70598981482908 s
FishTank20200327_154737 finished: 149.50173849286512 s
FishTank20200330_161100 finished: 161.42352586099878 s
FishTank20200331_162136 finished: 178.89885651692748 s
FishTank20200520_152810 finished: 203.18753069499508 s
FishTank20200521_154541 finished: 215.62737848400138 s
FishTank20200525_161602 finished: 228.13545213686302 s


/home/liam/code/Dynamics_of_dominance/tools/visualize_windowed_distributions/WindowedDistributionPlotter.py:390: RuntimeWarning: invalid value encountered in true_divide
  vec_ts = vec_ts / row_norms[:, np.newaxis]


FishTank20200526_160100 finished: 242.45791984791867 s
FishTank20200527_152401 finished: 253.32523146900348 s
FishTank20200824_151740 finished: 285.8628172748722 s
FishTank20200828_155504 finished: 302.1626975189429 s


/home/liam/code/Dynamics_of_dominance/tools/visualize_windowed_distributions/WindowedDistributionPlotter.py:95: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(nrows=nrows, ncols=1, figsize=(10,6), sharex=False)


FishTank20200902_160124 finished: 316.893324584933 s
FishTank20200903_160946 finished: 331.99565894086845 s

finished FishTank20200903_160946: 331.9961057950277 s


# Plot the bouts used for figure 7

In [13]:
t0 = time.perf_counter()

saveFolder = '/home/liam/temp/image_transfer/fight_bout_info'
if ~os.path.exists(saveFolder):
    os.makedirs(saveFolder, exist_ok=True)


for expIdx in range(len(expNames)):

    # plotting params
    expName = expNames[expIdx]                  # the name of the exp, for plot title
    figsavepath = os.path.join(saveFolder, 
                             f'{expName}.png')  # the filepath of the outputted png
    window_size=6000                            # the window width [frames]
    window_step=100                             # the step forward in time between windows [frames]
    dpp_bins = np.linspace(0, 30, 300)          # visualization bins
    tet_bins = np.linspace(-np.pi, np.pi, 300)  # visualization bins
    dpp_vmax=150                                # the max count color threshold when plotting.
    tet_vmax=150                                # the max count color threshold when plotting.
    num_xticks=10                               # the number of xaxis time ticks
    cmap='Blues'                                # colorscheme
    fps=100                                     # for frame-minutes conversions
    use_cbar=False                              # use/not-use the colorbar of the plots

    traj = smooth_trajectories[expIdx]
    expName = expNames[expIdx]
    winIdx = winner_idxs[expIdx]
    losIdx = loser_idxs[expIdx]
    has_clear_winner = conclusive_winner_loser[expIdx]
    exp_fight_bout_info = fight_bout_info[fight_bout_info[:,0]==expIdx][:,1:3]


    compute_dpp_tet_tet_distribution_plot_with_fight_bouts(traj, expName, exp_fight_bout_info, figsavepath, dpp_bins, tet_bins,
                                                           winnerIdx=winIdx, loserIdx=losIdx, has_clear_winner=has_clear_winner, 
                                                           window_size=window_size, window_step=window_step, dpp_vmax=dpp_vmax, 
                                                           tet_vmax=tet_vmax, num_xticks=num_xticks, cmap='Blues', fps=fps, use_cbar=use_cbar)
    print(f'{expName} finished: {time.perf_counter()-t0} s')


tE = time.perf_counter()
print()
print('finished {0}: {1} s'.format(expName, tE-t0))

FishTank20200127_143538 finished: 19.566051193978637 s
FishTank20200129_140656 finished: 36.18120930995792 s
FishTank20200130_153857 finished: 46.94131111493334 s
FishTank20200130_181614 finished: 58.080179505981505 s
FishTank20200207_161445 finished: 68.07872270094231 s
FishTank20200213_154940 finished: 82.98493374395184 s
FishTank20200214_153519 finished: 93.42429922288284 s
FishTank20200217_160052 finished: 106.00178869394585 s
FishTank20200218_153008 finished: 117.64050105400383 s
FishTank20200316_163320 finished: 137.86001128493808 s
FishTank20200327_154737 finished: 149.56159618892707 s
FishTank20200330_161100 finished: 161.35759116290137 s
FishTank20200331_162136 finished: 177.65423091407865 s
FishTank20200520_152810 finished: 201.69326737103984 s
FishTank20200521_154541 finished: 213.94117572391406 s
FishTank20200525_161602 finished: 226.37104637594894 s
FishTank20200526_160100 finished: 240.5831644369755 s
FishTank20200527_152401 finished: 251.17372226691805 s
FishTank20200824

# Plot the figure 7 bouts but without thresholding on duration (i.e. more fights than figure 7 bout set)

In [14]:
t0 = time.perf_counter()

saveFolder = '/home/liam/temp/image_transfer/fight_bout_info_noDurThresh'
if ~os.path.exists(saveFolder):
    os.makedirs(saveFolder, exist_ok=True)


for expIdx in range(len(expNames)):

    # plotting params
    expName = expNames[expIdx]                  # the name of the exp, for plot title
    figsavepath = os.path.join(saveFolder, 
                             f'{expName}.png')  # the filepath of the outputted png
    window_size=6000                            # the window width [frames]
    window_step=100                             # the step forward in time between windows [frames]
    dpp_bins = np.linspace(0, 30, 300)          # visualization bins
    tet_bins = np.linspace(-np.pi, np.pi, 300)  # visualization bins
    dpp_vmax=150                                # the max count color threshold when plotting.
    tet_vmax=150                                # the max count color threshold when plotting.
    num_xticks=10                               # the number of xaxis time ticks
    cmap='Blues'                                # colorscheme
    fps=100                                     # for frame-minutes conversions
    use_cbar=False                              # use/not-use the colorbar of the plots

    traj = smooth_trajectories[expIdx]
    expName = expNames[expIdx]
    winIdx = winner_idxs[expIdx]
    losIdx = loser_idxs[expIdx]
    has_clear_winner = conclusive_winner_loser[expIdx]
    exp_fight_bout_info = fight_bout_info_noDurThresh[fight_bout_info_noDurThresho[:,0]==expIdx][:,1:3]


    compute_dpp_tet_tet_distribution_plot_with_fight_bouts(traj, expName, exp_fight_bout_info, figsavepath, dpp_bins, tet_bins,
                                                           winnerIdx=winIdx, loserIdx=losIdx, has_clear_winner=has_clear_winner, 
                                                           window_size=window_size, window_step=window_step, dpp_vmax=dpp_vmax, 
                                                           tet_vmax=tet_vmax, num_xticks=num_xticks, cmap='Blues', fps=fps, use_cbar=use_cbar)
    print(f'{expName} finished: {time.perf_counter()-t0} s')


tE = time.perf_counter()
print()
print('finished {0}: {1} s'.format(expName, tE-t0))

FishTank20200127_143538 finished: 19.526265108026564 s
FishTank20200129_140656 finished: 35.510917973006144 s
FishTank20200130_153857 finished: 46.41872395505197 s
FishTank20200130_181614 finished: 58.64189301687293 s
FishTank20200207_161445 finished: 68.59135012002662 s
FishTank20200213_154940 finished: 83.36718784994446 s
FishTank20200214_153519 finished: 93.8108087410219 s
FishTank20200217_160052 finished: 106.34544194396585 s
FishTank20200218_153008 finished: 117.82138499803841 s
FishTank20200316_163320 finished: 137.94553469889797 s
FishTank20200327_154737 finished: 149.55468860291876 s
FishTank20200330_161100 finished: 161.34889133786783 s
FishTank20200331_162136 finished: 176.8106795400381 s
FishTank20200520_152810 finished: 201.3814470958896 s
FishTank20200521_154541 finished: 213.82960274000652 s
FishTank20200525_161602 finished: 226.31580829201266 s
FishTank20200526_160100 finished: 240.62033270602114 s
FishTank20200527_152401 finished: 251.36723829386756 s
FishTank20200824_1

In [15]:
exp_fight_bout_info

array([], shape=(0, 4), dtype=int64)

In [16]:
exp_fight_bout_info = fight_bout_info_noDurThresh[fight_bout_info_noDurThresh[:,0]==2]
exp_fight_bout_info

array([[     2,  31500, 175400, 143900],
       [     2, 301300, 319600,  18300]])